In [1]:
import pandas as pd
import numpy as np
import seaborn as sbn
import matplotlib.pyplot as plt


In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn import metrics
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_curve, confusion_matrix
from sklearn.preprocessing import MinMaxScaler

In [13]:
df= pd.read_csv("D:/kepler_data.csv")

In [14]:
df.head()

,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,10811496,K00753.01,NaN,CANDIDATE,CANDIDATE,0.000,0,0,0,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [16]:
df.shape

(9564, 49)

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9564 entries, 0 to 9563
Data columns (total 49 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   kepid              9564 non-null   int64  
 1   kepoi_name         9564 non-null   object 
 2   kepler_name        2360 non-null   object 
 3   koi_disposition    9564 non-null   object 
 4   koi_pdisposition   9564 non-null   object 
 5   koi_score          8054 non-null   float64
 6   koi_fpflag_nt      9564 non-null   int64  
 7   koi_fpflag_ss      9564 non-null   int64  
 8   koi_fpflag_co      9564 non-null   int64  
 9   koi_fpflag_ec      9564 non-null   int64  
 10  koi_period         9564 non-null   float64
 11  koi_period_err1    9110 non-null   float64
 12  koi_period_err2    9110 non-null   float64
 13  koi_time0bk        9564 non-null   float64
 14  koi_time0bk_err1   9110 non-null   float64
 15  koi_time0bk_err2   9110 non-null   float64
 16  koi_impact         9201 

In [18]:
df['ExoplanetCandidate'] = df['koi_pdisposition'].apply(lambda x: 1 if x == 'CANDIDATE' else 0)

In [19]:
df['ExoplanetConfirmed'] = df['koi_disposition'].apply(lambda x: 2 if x == 'CONFIRMED' else 1 if x == 'CANDIDATE' else 0 )

In [23]:
df.drop(columns=['kepler_name','kepoi_name','koi_teq_err1','koi_teq_err2',
                 'kepid','koi_disposition','koi_pdisposition',
                 'koi_fpflag_nt','koi_fpflag_ss','koi_fpflag_co',
                 'koi_fpflag_ec','koi_tce_delivname'], inplace=True) 

In [24]:
df.shape

(9564, 39)

In [27]:
df.isna().any()

koi_score              True
koi_period            False
koi_period_err1        True
koi_period_err2        True
koi_time0bk           False
koi_time0bk_err1       True
koi_time0bk_err2       True
koi_impact             True
koi_impact_err1        True
koi_impact_err2        True
koi_duration          False
koi_duration_err1      True
koi_duration_err2      True
koi_depth              True
koi_depth_err1         True
koi_depth_err2         True
koi_prad               True
koi_prad_err1          True
koi_prad_err2          True
koi_teq                True
koi_insol              True
koi_insol_err1         True
koi_insol_err2         True
koi_model_snr          True
koi_tce_plnt_num       True
koi_steff              True
koi_steff_err1         True
koi_steff_err2         True
koi_slogg              True
koi_slogg_err1         True
koi_slogg_err2         True
koi_srad               True
koi_srad_err1          True
koi_srad_err2          True
ra                    False
dec                 

In [28]:
df.dropna(inplace=True)

In [29]:
df.shape

(7803, 39)

In [30]:
def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred)))
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred)))
    
# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP')
    print(confusion_matrix(y_true, y_pred).ravel())
    
# Function Prints best parameters for GridSearchCV
def print_results(results):
    print('Best Parameters: {}\n'.format(results.best_params_)) 
    

In [31]:
features = df.drop(columns=['ExoplanetCandidate','ExoplanetConfirmed'])
target = df.ExoplanetCandidate

In [32]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1, test_size=.40)

In [35]:
lr = LogisticRegression(C=100, max_iter=200, class_weight='balanced')

# Fitting Model to the train set
lr.fit(X_train, y_train)

# Predicting on the test set
y_pred = lr.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.8167841127482383
Recall: 0.8574938574938575
F1 Score: 0.8299643281807373
Precision: 0.804147465437788

Confusion Matrix:
 TN,  FP, FN, TP
[1154  340  232 1396]


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


In [36]:
forest = RandomForestClassifier(n_estimators=100, criterion='gini')
# Fitting Model to the train set
forest.fit(X_train, y_train)
# Predicting on the test set
y_pred = forest.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

Evaluation Metrics:
Accuracy: 0.9606021780909674
Recall: 0.9465601965601965
F1 Score: 0.9616224648985959
Precision: 0.9771718452758402

Confusion Matrix:
 TN,  FP, FN, TP
[1458   36   87 1541]


For the data analysis part, first checked the null values present, dropped a few columns which weren't required for model training and then used the train test split to divide in training and testing part.
Tried using Logistic Regression and Random forest classifier, since this is a classification problem. Specified a function that calculates the various evaluation matrices score for a model. RF performs better than Logistic Regression.